## I had problems with my TensorFlow, and I founded this DataSet on Kaggle. So I work here, sorry!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/pokemon-images-and-types/pokemon.csv")
df.head()

In [ ]:
import os
from glob import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, regularizers
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
os.listdir('/kaggle/input/pokemon-images-and-types/images/images')

In [ ]:
os.listdir('/kaggle/input/pokemon-images-and-types/')

**Image Preprocessor**

In [ ]:
def img_preprocess():
    img_list = []

    for img_path in glob('/kaggle/input/pokemon-images-and-types/images/images/*.png'):
        img = cv.imread(img_path)           
        img = cv.resize(img, (224, 224))     
        sc_img = img.astype('float') / 255   
        img_list.append(sc_img)
    
    return np.asarray(img_list)


In [ ]:
img_arr = img_preprocess()

In [ ]:
plt.imshow(img_arr[11])
plt.xticks([])
plt.yticks([])

In [ ]:
img_arr.shape

**Model**

In [ ]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(filters=64,
           kernel_size=(3,3),
           padding='same', 
           activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2,2),
                 padding='same')(x)
x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2),
                 padding='same')(x)
x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
encoder_layer = MaxPooling2D(pool_size=(2,2),
                             padding='same')(x)

x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(encoder_layer)
x = UpSampling2D(size=(2,2))(x)         
x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = UpSampling2D(size=(2,2))(x)
x = Conv2D(filters=64,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = UpSampling2D(size=(2,2))(x)
decoder_layer = Conv2D(filters=3,
                       kernel_size=(3,3),
                       padding='same',
                       activation='relu')(x)

conv_auto_encoder = Model(input_layer, decoder_layer)

encoder = Model(input_layer, encoder_layer)

In [ ]:
conv_auto_encoder.summary()

In [ ]:
conv_auto_encoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
callbacks = EarlyStopping(monitor='val_loss',
                          min_delta=0.0001,
                          patience=5,
                          restore_best_weights=True)

In [ ]:
history = conv_auto_encoder.fit(img_arr, img_arr,
                                epochs=100, batch_size=256,
                                validation_split=0.3,
                                shuffle=True,
                                callbacks=[callbacks])

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

## Your notebook tried to allocate more memory than is available. It has restarted.
I don't have enough memory to compile it. ((

And I have "errors" in the next steps


In [ ]:
encoded = encoder.predict(img_arr)
decoded = conv_auto_encoder.predict(img_arr)

In [ ]:
print ('Low dimention shape: {}'.format(encoded[0].shape))
print ('output dimention shape: {}'.format(decoded[0].shape))

In [ ]:
fig = plt.figure(figsize=(28, 12))

n_plots = 10
n_row = int(n_plots / 2)

for j in range(n_plots):
    fig.add_subplot(n_row, 6, 3*j+1)
    plot_tmp = plt.imshow(img_arr[j])
    plt.xticks([])
    plt.yticks([])
    
    fig.add_subplot(n_row, 6, 3*j+2)
    plot_tmp = plt.imshow(encoded[j].reshape(32, 784))
    plt.xticks([])
    plt.yticks([])
    
    fig.add_subplot(n_row, 6, 3*j+3)
    plot_tmp = plt.imshow(decoded[j])
    plt.xticks([])
    plt.yticks([])
    
plt.show()